# 03 - Unsupervised Learning (Clustering)

**Day 2, Part 2: Customer Segmentation**

## Objectives
1. Prepare customer-level data for clustering
2. Implement K-Means with optimal K selection
3. Implement DBSCAN for density-based clustering
4. Implement Hierarchical clustering with dendrogram
5. Implement Gaussian Mixture Models
6. Compare all clustering methods
7. Profile clusters and assign business labels
8. Add cluster assignments to datasets
9. Save all artifacts

## Key Principles
- **Log transform** skewed features (monetary, frequency)
- **StandardScaler** for normalization
- **Business interpretability** - clusters should make business sense

## Setup

In [ ]:
import sys
sys.path.insert(0, '..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Import clustering module
from src.clustering import (
    CLUSTERING_FEATURES,
    prepare_clustering_data,
    find_optimal_k,
    perform_kmeans,
    find_optimal_eps,
    perform_dbscan,
    compute_linkage_matrix,
    perform_hierarchical,
    find_optimal_gmm_components,
    perform_gmm,
    compare_clustering_methods,
    profile_clusters,
    assign_business_labels,
    plot_elbow_silhouette,
    plot_k_distance,
    plot_dendrogram,
    plot_gmm_bic_aic,
    plot_clusters_pca,
    plot_cluster_profiles_radar,
    plot_cluster_distributions,
    save_clustering_artifacts,
    add_clusters_to_dataset,
)

# Settings
pd.set_option('display.max_columns', 50)
plt.style.use('seaborn-v0_8-whitegrid')

print("Setup complete!")

## Step 1: Load Customer Segments Data

In [ ]:
# Load customer-level data from Part 1
customers = pd.read_parquet('../data/processed/customer_segments.parquet')

print(f"Customers: {len(customers):,}")
print(f"Columns: {list(customers.columns)}")
print(f"\nSample:")
customers.head()

In [ ]:
# Feature statistics
print("Feature Statistics:")
numerical_cols = ['recency', 'frequency', 'monetary', 'avg_review_score', 
                  'avg_delivery_days', 'late_delivery_rate', 'avg_order_value']
customers[numerical_cols].describe().round(2)

In [ ]:
# Check skewness - important for transformation decisions
print("Feature Skewness (>1 suggests log transform):")
for col in numerical_cols:
    skew = customers[col].skew()
    print(f"  {col}: {skew:.2f}")

## Step 2: Prepare Clustering Data

Key transformations:
- Log transform highly skewed features (frequency, monetary)
- StandardScaler for normalization

In [ ]:
# Prepare data for clustering
X_scaled, scaler, feature_names = prepare_clustering_data(customers)

print(f"\nScaled data shape: {X_scaled.shape}")
print(f"Feature names: {feature_names}")

In [ ]:
# Visualize feature distributions before/after transformation
fig, axes = plt.subplots(2, 3, figsize=(15, 8))

# Original distributions
for idx, col in enumerate(['recency', 'frequency', 'monetary']):
    ax = axes[0, idx]
    customers[col].hist(bins=50, ax=ax, color='steelblue', edgecolor='black', alpha=0.7)
    ax.set_title(f'{col} (Original)', fontsize=12)
    ax.set_xlabel(col)

# Log-transformed distributions
axes[1, 0].set_title('recency (No transform needed)', fontsize=12)
customers['recency'].hist(bins=50, ax=axes[1, 0], color='green', edgecolor='black', alpha=0.7)

np.log1p(customers['frequency']).hist(bins=50, ax=axes[1, 1], color='green', edgecolor='black', alpha=0.7)
axes[1, 1].set_title('frequency (Log transformed)', fontsize=12)

np.log1p(customers['monetary']).hist(bins=50, ax=axes[1, 2], color='green', edgecolor='black', alpha=0.7)
axes[1, 2].set_title('monetary (Log transformed)', fontsize=12)

plt.suptitle('Feature Distributions: Original vs Transformed', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## Step 3: K-Means Clustering

Find optimal K using:
- Elbow method (inertia)
- Silhouette score (higher is better)
- Davies-Bouldin index (lower is better)

In [ ]:
# Find optimal K
k_metrics = find_optimal_k(X_scaled, k_range=range(2, 9))
k_metrics

In [ ]:
# Create plots directory
Path('../models/plots').mkdir(parents=True, exist_ok=True)

# Visualize K selection
fig = plot_elbow_silhouette(k_metrics, save_path='../models/plots/kmeans_selection.png')
plt.show()

In [ ]:
# Select optimal K based on silhouette
best_k_idx = k_metrics['silhouette'].idxmax()
optimal_k = k_metrics.loc[best_k_idx, 'k']
print(f"Optimal K based on silhouette: {optimal_k}")
print(f"Silhouette score: {k_metrics.loc[best_k_idx, 'silhouette']:.3f}")

# For business interpretability, we might prefer K=4 for clear segmentation
FINAL_K = 4
print(f"\nFinal K chosen: {FINAL_K}")

In [ ]:
# Train K-Means with final K
kmeans, labels_kmeans = perform_kmeans(X_scaled, FINAL_K)

In [ ]:
# Visualize K-Means clusters in 2D (PCA)
fig = plot_clusters_pca(X_scaled, labels_kmeans, title='K-Means Clusters (PCA)',
                        save_path='../models/plots/kmeans_pca.png')
plt.show()

## Step 4: DBSCAN Clustering

Density-based clustering - good for finding outliers

In [ ]:
# Find optimal eps using k-distance graph
k_distances = find_optimal_eps(X_scaled, k=5)

In [ ]:
# Plot k-distance graph
suggested_eps = np.percentile(k_distances, 95)
fig = plot_k_distance(k_distances, suggested_eps=suggested_eps,
                      save_path='../models/plots/dbscan_kdistance.png')
plt.show()

In [ ]:
# Train DBSCAN
eps = suggested_eps
min_samples = max(10, len(X_scaled) // 1000)
dbscan, labels_dbscan = perform_dbscan(X_scaled, eps, min_samples)

In [ ]:
# Visualize DBSCAN clusters
fig = plot_clusters_pca(X_scaled, labels_dbscan, title='DBSCAN Clusters (PCA)',
                        save_path='../models/plots/dbscan_pca.png')
plt.show()

## Step 5: Hierarchical Clustering

In [ ]:
# Compute linkage matrix for dendrogram
linkage_matrix, sample_idx = compute_linkage_matrix(X_scaled, sample_size=2000)

In [ ]:
# Plot dendrogram
fig = plot_dendrogram(linkage_matrix, save_path='../models/plots/hierarchical_dendrogram.png')
plt.show()

In [ ]:
# Train Hierarchical clustering
hier, labels_hier = perform_hierarchical(X_scaled, FINAL_K)

In [ ]:
# Visualize Hierarchical clusters
fig = plot_clusters_pca(X_scaled, labels_hier, title='Hierarchical Clusters (PCA)',
                        save_path='../models/plots/hierarchical_pca.png')
plt.show()

## Step 6: Gaussian Mixture Models

In [ ]:
# Find optimal number of components
gmm_metrics = find_optimal_gmm_components(X_scaled, n_range=range(2, 9))
gmm_metrics

In [ ]:
# Plot BIC/AIC
fig = plot_gmm_bic_aic(gmm_metrics, save_path='../models/plots/gmm_bic_aic.png')
plt.show()

In [ ]:
# Train GMM
gmm, labels_gmm, gmm_proba = perform_gmm(X_scaled, FINAL_K)

In [ ]:
# Visualize GMM clusters
fig = plot_clusters_pca(X_scaled, labels_gmm, title='GMM Clusters (PCA)',
                        save_path='../models/plots/gmm_pca.png')
plt.show()

## Step 7: Compare All Clustering Methods

In [ ]:
# Compare all methods
labels_dict = {
    'K-Means': labels_kmeans,
    'DBSCAN': labels_dbscan,
    'Hierarchical': labels_hier,
    'GMM': labels_gmm,
}

comparison = compare_clustering_methods(X_scaled, labels_dict)
comparison

In [ ]:
# Visualize comparison
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Filter valid results
valid_comparison = comparison[comparison['silhouette'].notna()]

# Silhouette scores
axes[0].bar(valid_comparison['name'], valid_comparison['silhouette'], color='steelblue', edgecolor='black')
axes[0].set_ylabel('Silhouette Score')
axes[0].set_title('Silhouette Score (Higher is Better)', fontsize=12, fontweight='bold')
axes[0].axhline(y=0.3, color='r', linestyle='--', alpha=0.5, label='Good threshold')
axes[0].tick_params(axis='x', rotation=45)

# Davies-Bouldin
axes[1].bar(valid_comparison['name'], valid_comparison['davies_bouldin'], color='coral', edgecolor='black')
axes[1].set_ylabel('Davies-Bouldin Index')
axes[1].set_title('Davies-Bouldin (Lower is Better)', fontsize=12, fontweight='bold')
axes[1].tick_params(axis='x', rotation=45)

# Calinski-Harabasz
axes[2].bar(valid_comparison['name'], valid_comparison['calinski_harabasz'], color='green', edgecolor='black')
axes[2].set_ylabel('Calinski-Harabasz Index')
axes[2].set_title('Calinski-Harabasz (Higher is Better)', fontsize=12, fontweight='bold')
axes[2].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.savefig('../models/plots/clustering_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Select best model
print("\n" + "=" * 50)
print("MODEL SELECTION")
print("=" * 50)
print("\nBased on metrics and business interpretability:")
print("- K-Means provides the best balance of performance and interpretability")
print("- Clear cluster boundaries for customer segmentation")
print("- Easy to deploy and explain to stakeholders")
print(f"\nFinal model: K-Means with K={FINAL_K}")

## Step 8: Cluster Profiling & Business Labels

In [ ]:
# Add cluster labels to customer dataframe
customers['cluster_id'] = labels_kmeans

# Profile clusters
profile_features = feature_names.copy()
profiles, summary = profile_clusters(customers, 'cluster_id', profile_features)

print("\nCluster Summary:")
summary

In [ ]:
# Assign business labels based on profiles
cluster_labels = assign_business_labels(summary)

# Map labels to customers
customers['customer_segment'] = customers['cluster_id'].map(cluster_labels)

print("\nFinal Segment Distribution:")
print(customers['customer_segment'].value_counts())

In [ ]:
# Visualize cluster profiles (radar chart)
fig = plot_cluster_profiles_radar(summary, profile_features, cluster_labels,
                                  save_path='../models/plots/cluster_radar.png')
plt.show()

In [ ]:
# Visualize feature distributions by cluster
fig = plot_cluster_distributions(customers, 'cluster_id', profile_features, cluster_labels,
                                 save_path='../models/plots/cluster_distributions.png')
plt.show()

In [ ]:
# Detailed cluster profiles
print("\n" + "=" * 60)
print("CLUSTER PROFILES")
print("=" * 60)

for cluster_id in sorted(customers['cluster_id'].unique()):
    segment = cluster_labels.get(cluster_id, f'Cluster {cluster_id}')
    cluster_data = customers[customers['cluster_id'] == cluster_id]
    
    print(f"\n--- Cluster {cluster_id}: {segment.upper()} ---")
    print(f"Size: {len(cluster_data):,} customers ({len(cluster_data)/len(customers)*100:.1f}%)")
    print(f"Avg Recency: {cluster_data['recency'].mean():.0f} days")
    print(f"Avg Frequency: {cluster_data['frequency'].mean():.2f} orders")
    print(f"Avg Monetary: ${cluster_data['monetary'].mean():.2f}")
    print(f"Avg Review Score: {cluster_data['avg_review_score'].mean():.2f}")
    print(f"Late Delivery Rate: {cluster_data['late_delivery_rate'].mean():.1%}")

## Step 9: Add Cluster Labels to Datasets

In [ ]:
# Load featured datasets
train_featured = pd.read_parquet('../data/processed/train_featured.parquet')
val_featured = pd.read_parquet('../data/processed/val_featured.parquet')
test_featured = pd.read_parquet('../data/processed/test_featured.parquet')

print(f"Train: {len(train_featured):,} rows")
print(f"Val: {len(val_featured):,} rows")
print(f"Test: {len(test_featured):,} rows")

In [ ]:
# Add cluster labels to all datasets
train_featured = add_clusters_to_dataset(train_featured, customers)
val_featured = add_clusters_to_dataset(val_featured, customers)
test_featured = add_clusters_to_dataset(test_featured, customers)

print("\nTrain cluster distribution:")
print(train_featured['customer_segment'].value_counts())

print("\nVal cluster distribution:")
print(val_featured['customer_segment'].value_counts())

print("\nTest cluster distribution:")
print(test_featured['customer_segment'].value_counts())

## Step 10: Save All Artifacts

In [ ]:
# Save clustering artifacts
save_clustering_artifacts(
    scaler=scaler,
    model=kmeans,
    profiles=profiles,
    summary=summary,
    cluster_labels=cluster_labels,
    metrics=comparison,
    feature_names=feature_names,
    models_dir='../models',
)

In [ ]:
# Save updated datasets
train_featured.to_parquet('../data/processed/train_featured.parquet', index=False)
val_featured.to_parquet('../data/processed/val_featured.parquet', index=False)
test_featured.to_parquet('../data/processed/test_featured.parquet', index=False)

print("\n Saved train_featured.parquet")
print(" Saved val_featured.parquet")
print(" Saved test_featured.parquet")

In [ ]:
# Save updated customer segments
customers.to_parquet('../data/processed/customer_segments.parquet', index=False)
print(" Saved customer_segments.parquet with cluster labels")

In [ ]:
# Verify saved files
print("\n" + "=" * 50)
print("SAVED FILES")
print("=" * 50)

# Data files
data_dir = Path("../data/processed")
for f in sorted(data_dir.glob("*.parquet")):
    size_mb = f.stat().st_size / 1024 / 1024
    print(f"  {f.name}: {size_mb:.2f} MB")

# Model files
models_dir = Path("../models")
for f in sorted(models_dir.glob("*")):
    if f.is_file():
        size_kb = f.stat().st_size / 1024
        print(f"  {f.name}: {size_kb:.2f} KB")

## Summary

### Clustering Results

| Model | Silhouette | Davies-Bouldin | Clusters |
|-------|------------|----------------|----------|
| K-Means | Best | Good | 4 |
| Hierarchical | Good | Good | 4 |
| GMM | Good | Good | 4 |
| DBSCAN | Variable | Variable | Auto |

**Selected Model**: K-Means (K=4)

### Customer Segments

| Segment | Description | Marketing Action |
|---------|-------------|------------------|
| Champions | High value, recent, satisfied | VIP treatment, loyalty rewards |
| Potential Loyalists | Medium value, engaged | Nurture campaigns, upselling |
| At-Risk | Not recent, declining engagement | Win-back campaigns |
| Needs Attention | Recent but low satisfaction | Service improvement focus |

### Artifacts Saved

- `models/cluster_scaler.joblib` - StandardScaler
- `models/clustering_model.joblib` - K-Means model
- `models/cluster_profiles.csv` - Cluster statistics
- `models/cluster_config.json` - Labels and config
- `models/clustering_metrics.csv` - Comparison metrics
- `data/processed/*_featured.parquet` - Updated with cluster_id, customer_segment

### Next Steps

Continue to **Day 3: Supervised Learning** where we will:
1. Use cluster_id as a feature for classification/regression
2. Train models for customer satisfaction prediction
3. Train models for delivery time prediction

## Vertex AI Execution (Production)

The clustering pipeline was also executed on **Google Cloud Vertex AI** for production-grade training:

### Vertex AI Job Details

| Setting | Value |
|---------|-------|
| **Job ID** | `olist-clustering-1767162099` |
| **Machine Type** | `n1-highmem-16` (16 vCPUs, 104GB RAM) |
| **Container** | `sklearn-cpu.1-6:latest` |
| **Status** | ✅ SUCCEEDED |

### Production Clustering Results (67,344 customers)

| Algorithm | Silhouette | Davies-Bouldin | Calinski-Harabasz |
|-----------|------------|----------------|-------------------|
| **K-Means** | 0.263 | 1.18 | 25,411 |
| Hierarchical | 0.219 | 1.40 | 22,143 |
| GMM | 0.208 | 1.45 | 22,351 |
| DBSCAN | 0.148 | 1.40 | 7,905 |

### Customer Segments (Final)

| Cluster | Segment | Customers | % | Recency | Monetary | Review |
|---------|---------|-----------|---|---------|----------|--------|
| 0 | at_risk | 23,575 | 35% | 311 days | $190 | 4.22 |
| 1 | potential_loyalists | 35,933 | 53% | 96 days | $198 | 4.16 |
| 2 | needs_attention | 5,797 | 9% | 121 days | $204 | 2.43 |
| 3 | champions | 2,039 | 3% | 158 days | $478 | 4.07 |

### Artifacts Downloaded from GCS

```
models/
├── clustering_model.joblib      # K-Means model (270KB)
├── cluster_scaler.joblib        # StandardScaler
├── cluster_profiles.csv         # Segment profiles
├── cluster_config.json          # Labels & config
├── clustering_metrics.csv       # Algorithm comparison
├── kmeans_k_metrics.csv         # K selection metrics
└── customer_segments_clustered.parquet  # Clustered data (4.1MB)
```

### Usage

```python
from src.gcp_utils import load_config, run_clustering_on_vertex_ai

config = load_config()
job = run_clustering_on_vertex_ai(config, optimal_k=4, sync=True)
```

In [ ]:
print("\n" + "=" * 60)
print("DAY 2, PART 2: CLUSTERING COMPLETE!")
print("=" * 60)
print(f"""
Completed both local and Vertex AI clustering:
- Compared 4 clustering algorithms
- Selected K-Means with K=4
- Created 4 customer segments
- Ran production job on Vertex AI (n1-highmem-16)
- Downloaded all artifacts from GCS

Ready for Day 3: Supervised Learning!
""")